# <font color=green>Restaurant-Guru

> Este projeto tem como finalidade testar os conhecimentos que obtive completando a formação "Python para Data Science" da plataforma Alura. Aqui irei testar meus conhecimentos acerca de Web Scrapping e EDA (Exploratory Data Analysis).

<h4>Ideia Central:</h4> 

> A ideia do projeto é fazer um Web Scrapping do site https://restaurantguru.com.br/ e conseguir extrair informações valiosas por meio da análise de dados, gráficos e etc.

<h4>Sobre o site:</h4> 

> Restaurant Guru é um wiki+metasearch para restaurantes. Eles fornecem aos usuários informações completas sobre restaurantes como: avaliações dos visitantes, cardápios, fotos, etc. O principal objetivo do site é ajudar pessoas a escolherem o lugar certo para comer, seja em sua cidade ou durante sua viagem.

## <font color=blue>Importando Bibliotecas

> Neste passo iremos importar todas as bibliotecas que iremos utilizar nesse projeto

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from urllib.request import urlopen, urlretrieve, Request
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup

## <font color=blue>Web Scrapping

<img src='img/extrair.png'>

A imagem acima mostra os campos que iremos extrair do site. Vamos obter o **nome**, a **localização**, o **tipo de comida**, as **avaliações** e o **valor** de cada restaurante.

#### Configurações básicas - Obtenção de URL

In [2]:
url = 'https://restaurantguru.com.br/restaurant-Brazil-t1'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

req = Request(url, headers = headers)
response = urlopen(req)
html = response.read()

#### Configurações básicas - Tratamento do HTML



> O tipo da variável *html* é **bytes**, executando o comando *html.decode('utf-8')* irei corrigir a acentuação de palavras e converter essa variável para **str**.

In [3]:
type(html)

bytes

In [4]:
html = html.decode('utf-8')

In [5]:
type(html)

str

>Elaboração de uma função para eliminar os caracteres de tabulação, quebra de linha e espaçamento entre TAGS. Após isso, foi aplicado a variável *html* para ser tratada.

In [6]:
def trata_html(input):
    return " ".join(html.split()).replace('> <', '><')

In [7]:
html = trata_html(html)

#### Criando um objeto BeautifulSoup

In [8]:
soup = BeautifulSoup(html, 'html.parser')

In [9]:
type(soup)

bs4.BeautifulSoup

#### Criando variávels para armazenar informações

In [10]:
cards = []
card = {}

### <font color=#3498DB>Obtendo os dados do primeiro CARD

In [11]:
anuncio = soup.find('div', {'class': 'restaurant_row show'})

**Obtendo o <font color=red>NOME</font> do estabelecimento:**

In [12]:
# Obtendo o título
nome = anuncio.find('div', {'class': 'title'}).get_text()
nome

'Figueira Rubayat / Restaurante, Churrascaria'

In [13]:
# Removendo o subtítulo
nome = nome.split(' /')[0]
nome

'Figueira Rubayat'

#### <font color=red> Resumo:

In [14]:
card['nome'] = nome

**Obtendo o <font color=red>LOCALIZAÇÃO</font> do estabelecimento:**

In [15]:
local = anuncio.find('div', {'class': 'number'}).get_text()
local

'#1 de 153705 lugar para comer em São Paulo'

In [16]:
local = local.split()
local

['#1', 'de', '153705', 'lugar', 'para', 'comer', 'em', 'São', 'Paulo']

In [17]:
local = local[7:]
local

['São', 'Paulo']

In [18]:
local = [' '.join(local)]
local

['São Paulo']

#### <font color=red> Resumo:

In [19]:
card['local'] = local

In [20]:
card

{'nome': 'Figueira Rubayat', 'local': ['São Paulo']}

**Obtendo o <font color=red>TIPOS DE COMIDA</font> do estabelecimento:**

In [21]:
comida = anuncio.find('div', {'class': 'info cuisine'}).get_text()
comida

'Brasileira, Argentina, Grelhada, Churrascarias'

In [22]:
comida = comida.split(', ')
comida

['Brasileira', 'Argentina', 'Grelhada', 'Churrascarias']

#### <font color=red> Resumo:

In [23]:
card['comida'] = comida

**Obtendo o <font color=red>VALOR</font> do estabelecimento:**

In [24]:
valor = anuncio.find('span', {'class': 'cost'}).get_text()
valor

'$$$$'

#### <font color=red> Resumo:

In [25]:
if valor == '$$$$':
    card['valor'] = 'Muito Caro'
elif valor == '$$$':
    card['valor'] = 'Caro'
elif valor == '$$':
    card['valor'] = 'Moderado'
elif valor == '$':
    card['valor'] = 'Barato'
else:
    card['valor'] = 0

**Obtendo o <font color=red>AVALIAÇÃO</font> do estabelecimento:**